In [ ]:
import pandas as pd 
import numpy as np 

import matplotlib.pyplot as plt 
import seaborn as sns; sns.set(style='ticks', color_codes=True)

from sklearn.model_selection import train_test_split
from sklearn.ensemble import ExtraTreesRegressor, RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.feature_selection import RFE
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import KFold

import os 

In [ ]:
print(os.listdir("../walmart-recruiting-store-sales-forecasting"))

In [ ]:
df_stores       = pd.read_csv("../walmart-recruiting-store-sales-forecasting/stores.csv", sep=',',header=0)
df_features     = pd.read_csv("../walmart-recruiting-store-sales-forecasting/features.csv", sep=',',header=0)
df_samplesub    = pd.read_csv("../walmart-recruiting-store-sales-forecasting/sampleSubmission.csv", sep=',',header=0)
df_train        = pd.read_csv("../walmart-recruiting-store-sales-forecasting/train.csv", sep=',',header=0)
df_test         = pd.read_csv("../walmart-recruiting-store-sales-forecasting/test.csv", sep=',',header=0)

In [ ]:
df_train_stores_features = df_train.merge(df_stores,how='left').merge(df_features,how='left')

In [ ]:
df_train_stores_features['Date'] = pd.to_datetime(df_train_stores_features['Date'])

In [ ]:
df_train_stores_features = df_train_stores_features.drop(columns=['Type'])

In [ ]:
df_train_stores_features

In [ ]:
df_stores

In [ ]:
df_stores.isnull().sum()

In [ ]:
df_features

In [ ]:
df_features.isnull().sum()

In [ ]:
df_samplesub

In [ ]:
df_samplesub.isnull().sum()

## Explorando os Dados

In [ ]:
df_train_stores_features.head()

In [ ]:
def scatter(df_train_stores_features, column):
    plt.figure()
    plt.scatter(df_train_stores_features[column], df_train_stores_features['Weekly_Sales'])
    plt.ylabel('weeklySales')
    plt.xlabel(column)

In [ ]:
df_train_stores_features.columns

In [ ]:
scatter(df_train_stores_features, 'Fuel_Price')
scatter(df_train_stores_features, 'Size')
scatter(df_train_stores_features, 'CPI')
scatter(df_train_stores_features, 'Type')
scatter(df_train_stores_features, 'IsHoliday')
scatter(df_train_stores_features, 'Unemployment')
scatter(df_train_stores_features, 'Temperature')
scatter(df_train_stores_features, 'Store')
scatter(df_train_stores_features, 'Dept')


In [ ]:
fig = plt.Figure(figsize=(18,14))
corr = df_train_stores_features.corr()
c = plt.pcolor(corr)
plt.yticks(np.arange(0.5, len(corr.index), 1), corr.index)
plt.xticks(np.arange(0.5, len(corr.columns), 1), corr.columns)
fig.colorbar(c)

In [ ]:
df_train_stores_features.columns

In [ ]:
sns.pairplot(df_train_stores_features,vars=['Weekly_Sales', 'Fuel_Price', 'Size', 'CPI', 'Dept', 'Temperature', 'Unemployment'])

In [ ]:
#sns.pairplot(df_train_stores_features.fillna(0),vars=['Weekly_Sales', 'MarkDown1', 'MarkDown2', 'MarkDown3', 'MarkDown4', 'MarkDown5'])

In [ ]:
for name, group in df_train_stores_features.groupby(["Store","Dept"]):
    plt.title(name)
    plt.scatter(range(len(group)), group["Weekly_Sales"])
    plt.show()
    break

## Manipulando Dados

In [ ]:
df_train_stores_features = pd.get_dummies(df_train_stores_features, columns=["Type"])
df_train_stores_features[['MarkDown1','MarkDown2','MarkDown3','MarkDown4','MarkDown5']] = df_train_stores_features[['MarkDown1','MarkDown2','MarkDown3','MarkDown4','MarkDown5']].fillna(0)
df_train_stores_features['Month'] = pd.to_datetime(df_train_stores_features['Date']).dt.month
df_train_stores_features = df_train_stores_features.drop(columns=['Date','CPI','Fuel_Price','Unemployment','MarkDown3'])

In [ ]:
df_train_stores_features

## Algoritimos

In [39]:
def knn():
    knn = KNeighborsRegressor(n_neighbors=10)
    return knn

def extraTreesRegressor():
    clf = ExtraTreesRegressor(n_estimators=100, max_features='auto', verbose=1, n_jobs=1)
    return clf

def randomForestRegressor():
    clf = RandomForestRegressor(n_estimators=100 , max_features='log2', verbose=1)
    return clf

def svm():
    clf = SVR(kernel='rgb', gamma='auto')
    return clf

def nn():
    clf = MLPRegressor(hidden_layer_sizes=(10,), activation='relu', verbose=3)
    return clf

def predict_(m, test_x):
    return pd.Series(m.predict(test_x))

def model_():
    return extraTreesRegressor()

def train_(train_x, train_y):
    m = model_()
    m.fit(train_x, train_y)
    return m

def train_and_predict(train_x, train_y, test_x):
    m = train_(train_x, train_y)
    return predict_(m, test_x), m

In [40]:
def calculate_error(test_y, predicted, weights):
    return mean_absolute_error(test_y, predicted, sample_weight=weights)

## K-Fold Cross Validation

In [41]:
kf = KFold(n_splits=5)
splited = []

for name, group in df_train_stores_features.groupby(['Store', 'Dept']):
    group = group.reset_index(drop=True)
    trains_x = []
    trains_y = []
    tests_x = []
    tests_y = []
    if group.shape[0] <= 5:
        f = np.array(range(5))
        np.random.shuffle(f)
        group['fold'] = f[:group.shape[0]]
        continue
    fold = 0
    for train_index, test_index in kf.split(group):
        group.loc[test_index, 'fold'] = fold
        fold += 1
    splited.append(group)

splited = pd.concat(splited).reset_index(drop=True)

In [38]:
splited

,Store,Dept,Weekly_Sales,IsHoliday,Size,Temperature,MarkDown1,MarkDown2,MarkDown4,MarkDown5,Type_A,Type_B,Type_C,Month,fold
0,1,1,24924.50,False,151315,42.31,0.00,0.00,0.00,0.00,True,False,False,2,0.0
1,1,1,46039.49,True,151315,38.51,0.00,0.00,0.00,0.00,True,False,False,2,0.0
2,1,1,41595.55,False,151315,39.93,0.00,0.00,0.00,0.00,True,False,False,2,0.0
3,1,1,19403.54,False,151315,46.63,0.00,0.00,0.00,0.00,True,False,False,2,0.0
4,1,1,21827.90,False,151315,46.50,0.00,0.00,0.00,0.00,True,False,False,3,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
421269,45,98,508.37,False,118221,64.88,4556.61,20.64,1601.01,3288.25,False,True,False,9,4.0
421270,45,98,628.10,False,118221,64.89,5046.74,0.00,2253.43,2340.01,False,True,False,10,4.0
421271,45,98,1061.02,False,118221,54.47,1956.28,0.00,599.32,3990.54,False,True,False,10,4.0
421272,45,98,760.01,False,118221,56.47,2004.02,0.00,437.73,1537.49,False,True,False,10,4.0


In [43]:
best_model = None
error_cv = 0
best_error = np.iinfo(np.int32).max
for fold in range(5):
    df_train = splited.loc[splited['fold'] != fold]
    df_test = splited.loc[splited['fold'] == fold]
    train_y = df_train['Weekly_Sales']
    train_x = df_train.drop(columns=['Weekly_Sales', 'fold'])
    test_y = df_test['Weekly_Sales']
    test_x = df_test.drop(columns=['Weekly_Sales', 'fold'])
    print(df_train.shape, df_test.shape)
    predicted, model = train_and_predict(train_x, train_y, test_x)
    weights = test_x['IsHoliday'].replace(True,5).replace(False, 1)
    error = calculate_error(test_y, predicted, weights)
    error_cv += error
    print(fold, error)
    if error < best_error:
        print('Find Best Model: ')
        best_error = error
        best_model = model
error_cv /= 5

(335722, 15) (85552, 15)


InvalidParameterError: The 'max_features' parameter of ExtraTreesRegressor must be an int in the range [1, inf), a float in the range (0.0, 1.0], a str among {'sqrt', 'log2'} or None. Got 'auto' instead.